In [1]:
import anndata
import pandas as pd
import numpy as np
import json 

from config import FDIR_EXTERNAL, FDIR_INTEMEDIATE, FDIR_PROCESSED, FDIR_RAW, PROJ_ROOT


In [29]:

fdir_raw = FDIR_RAW
fdir_intermediate = FDIR_INTEMEDIATE
fdir_processed = FDIR_PROCESSED / "sex"
fdir_external = FDIR_EXTERNAL
ml_models_fdir = PROJ_ROOT/"models"


use_CV = True

model_type = 'catboost'
model_type = 'xgboost'


value_to_predict = 'sex'

organ_names = {
    "BRAIN0": "BRAIN0",
    "HEART": "HEART",
    "BRAIN1": "BRAIN1",
    "None": "geuvadis",
    "CAGE.HEART": "CAGE.HEART",
}

filename_prefixes = {
    "None": "GEUVADIS",
    "CAGE.HEART": "CAGE.HEART",
    "BRAIN0": "BRAIN0",
    "HEART": "HEART",
    "BRAIN1": "BRAIN1",
}


In [34]:
n_transcripts = {}

n_train_transcripts = {}

for organ_name in ['None', "HEART", 'BRAIN0',  "BRAIN1"]:
    n_transcripts[organ_name] = {}
    n_train_transcripts[organ_name] = {}
    for sex_chromosome in ["chr_aXY",  "autosomes", "chr_aX", "chr_aY"]:
        adata = anndata.read_h5ad(fdir_processed / f"{filename_prefixes[organ_name]}.preprocessed.{value_to_predict}.h5ad")
        # adata = adata[:, adata.varm[sex_chromosome]]
        n_transcripts[organ_name][sex_chromosome] = sum(adata.varm[sex_chromosome])
        
        features_fname = (
            f"{organ_names[organ_name]}_train_features_{sex_chromosome}_calibration_{organ_name}.csv"
        )
        features_list = pd.read_csv(ml_models_fdir / model_type / features_fname, index_col=0)
        n_train_transcripts[organ_name][sex_chromosome] = len(features_list)

In [35]:
pd.DataFrame.from_dict(n_transcripts).T

,chr_aXY,autosomes,chr_aX,chr_aY
None,9526,9222,9469,9272
HEART,102378,99338,102278,99338
BRAIN0,151941,147125,151466,147392
BRAIN1,146083,141307,145636,141562


In [37]:
pd.DataFrame.from_dict(n_train_transcripts)

,None,HEART,BRAIN0,BRAIN1
chr_aXY,3,5,3,2
autosomes,40,40,46,29
chr_aX,5,4,23,4
chr_aY,2,35,2,4


In [11]:
sum(adata.varm[sex_chromosome])

9222